In [17]:
import os
ROOT = '/home/mav204/Documents/minor-project'
os.chdir(ROOT)
print(ROOT)

/home/mav204/Documents/minor-project


In [18]:
from math import ceil
from misc.dataset_modifier import separate, save_json, get_json
from augment.back_translation import Backtranslation
from augment.synonym_replacement import SynonymAugmenter

misc.dataset_modifier 




/home/mav204/Documents/minor-project/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to /home/mav204/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/mav204/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
BT = Backtranslation()
SYN = SynonymAugmenter()

Device:  cpu


In [20]:
train_data = get_json(f'./data/curated/train.json')
questions = get_json(f'./data/metadata/acceptable.json')

correct_baseline = 50
partially_correct_baseline = 50
incorrect_baseline = 25

lookup_table = {}

for q in questions:
    qc = questions[q]['count']  # shorthand

    c = qc['correct']
    p = qc['partially correct']
    i = qc['incorrect']

    lookup_table[q] = {
        'correct': ceil(max(0, correct_baseline - c) / c),
        'partially correct': ceil(max(0, partially_correct_baseline - p) / p),
        'incorrect': ceil(max(0, incorrect_baseline - i) / i)
    }


In [21]:
qid = ''
no_of_augmentations = 0
sim_ub = .85
sim_lb = .7
augmentations = []

In [ ]:
temp = []
count = 0
for record in train_data:
    qid = record['id'][-4:]
    num_augments = lookup_table[qid][record['verification_feedback']]

    for i in range(num_augments):
        answer = record['provided_answer']
        variant = BT.translate(answer)
        sim1 = SYN.semantic_sim(answer, variant)
        
        for j in range(1,3):
            if  sim1 < sim_lb  or sim1 > sim_ub:
                print(f'variant too similar, Re-translating...({j}/2)')
                variant = BT.translate(variant)
                sim1 = SYN.semantic_sim(answer, variant)
            else: break


        variant2 = SYN.augment(answer)

        sim2 = SYN.semantic_sim(answer, variant2)

        print(f'Back-Translation: {sim1} | Synonym replacement : {sim2}')

        v1_score = SYN.deviation_score(sim1)
        v2_score = SYN.deviation_score(sim2)

        best_variant = variant if v1_score < v2_score else variant2
        temp.append(best_variant)
        count += 1
    print(f'Number of augmentations generated: {count}'+
            f'Question Number: {record['id']}'+
            f'Classification: {record['verification_feedback']}'+
            '====================================================\n\n')
    
    for i in range(len(temp)):
        augmentations.append({
            'id': f'aug{i:02}'+record['id'],
            'question': record['question'],
            'reference_answer': record['reference_answer'],
            'provided_answer': temp[i],
            'answer_feedback': record['answer_feedback'],
            'verification_feedback': record['verification_feedback'],
            'max_score': record['max_score'],
            'normalized_score': record['normalized_score']
        })
    save_json(data=augmentations,
              path='./data/augmented/train.json',
              ask=False)
    
print(f'Augmentation length: {len(augmentations)}')


Generating Translation...
Augmentation succeeded after 13 attempts, similarity=0.840
sim1: 0.9423956274986267 | sim2 : 0.8397572040557861
1.0923956274986266 0.010242795944213845
Generating Translation...
Augmentation succeeded after 1 attempts, similarity=0.791
sim1: 0.8199583888053894 | sim2 : 0.7909116148948669
0.030041611194610573 0.059088385105133034
Generating Translation...
Augmentation succeeded after 1 attempts, similarity=0.774
sim1: 0.8578218221664429 | sim2 : 0.7735936045646667
1.0078218221664428 0.07640639543533323
Generating Translation...
Max attempts exceeded. Returning fallback.
sim1: 0.8578218221664429 | sim2 : 0.007918257266283035
1.0078218221664428 1.692081742733717
Generating Translation...
sim1: 1.0000001192092896 | sim2 : 0.20346291363239288
1.1500001192092895 1.496537086367607
Generating Translation...
Max attempts exceeded. Returning fallback.
sim1: 0.9384239912033081 | sim2 : 0.052025798708200455
1.088423991203308 1.6479742012917995
Generating Translation...
Au